# INM702 Neural Network for MNIST

In [12]:
import pandas as pd,numpy as np,matplotlib.pyplot as plt, os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
%matplotlib inline

## Mathematical Functions and Derivatives

In [63]:
#Activation Functions
import numpy as np
#Input and Hidden Layers
@np.vectorize
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

@np.vectorize
def d_sigmoid(x):
    s = sigmoid(x)
    return s * (1 - s)

@np.vectorize
def relu(x):
    return x * (x > 0)

@np.vectorize
def d_relu(x):
    return 1 * (x > 0)

#Output Layer
def softmax(x):
    e = np.exp(x)
    return e / e.sum()
    
#Loss Functions
@np.vectorize
def cross_entropy_loss(y_pred, y):
    if y == 1:
        return -np.log(y_pred)
    else:
        return -np.log(1 - y_pred)

#d_loss = dL/dsoftmax * dsoftmax/dz
@np.vectorize
def d_loss(y_pred, y):
    return y_pred - y
    

#Truncated Normal Distribution for Initialising weights
from scipy.stats import truncnorm
def trunc_norm(mean = 0, sd = 1, lower = -1, upper = 1):
    return truncnorm((lower - mean) / sd, (upper - mean) / sd, loc=mean, scale=sd)
    

## Neural Network Class

In [58]:
class NeuralNetwork:
    def __init__(self, hidden_layer_sizes = [128,128], learning_rate = 0.001 , activation_fn = 'r', dropout_rate = 0):
        #list for number of units in each layer, 28*28 inputs, 10 outputs
        self.layer_sizes = [28*28] + hidden_layer_sizes + [10]
        self.lr = learning_rate
        self.dr = dropout_rate
        
        #activation_fn parameter takes 'r'(default) or 's' to choose ReLu or Sigmoid for the activation function
        if activation_fn == 's':
            self.activation = sigmoid
            self.d_activation = d_sigmoid
        else:
            self.activation = relu
            self.d_activation = d_relu
            
        self.initialise_weights()
    
    def initialise_weights(self):
        self.weights = list()
        for i in range(len(self.layer_sizes) - 1):
            #use truncated normal distribution to intialise weights
            trunc = 1 / np.sqrt(self.layer_sizes[i])
            X = trunc_norm(mean = 0, sd = 1, lower = -trunc, upper = trunc)
            w = X.rvs((self.layer_sizes[i+1], self.layer_sizes[i]))
            self.weights.append(w)
    
    def forward_pass(self, x):
        x = np.array(x).T
        #input to hidden layer
        self.z = [np.dot(self.weights[0], x)]
        self.a = [self.activation(self.z[0])]
        
        #hidden layers
        for i in range(len(self.weights) - 2):
            self.z.append(np.dot(self.weights[1 + i], self.a[i]))
            self.a.append(self.activation(self.z[1 + i]))
            
        #hidden to output layer
        self.z.append(np.dot(self.weights[-1], self.a[-1]))
        self.a.append(softmax(self.z[-1]))
        
    
    def back_prop(self, y):
        pass
    
    def train(self, x_train, y_train, epochs = 5):
        pass
    
    def test(self, x_test):
        pass
            
        

## Load and Preprocess MNIST dataset

In [15]:
#Load train and test csv
input_folder_path = "MNIST_data/"

train_df = pd.read_csv(input_folder_path + "mnist_train.csv")
test_df = pd.read_csv(input_folder_path + "mnist_test.csv")

#First column is the target/label
train_labels = train_df['label'].values
test_labels = test_df['label'].values

#Pixels values start from the 2nd column
train_images = (train_df.iloc[:,1:].values).astype('float32')
test_images = (test_df.iloc[:,1:].values).astype('float32')

#Normalise
train_images /= 255
test_images /=255

#One Hot Encoding
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

## Create NN instance

In [67]:
simple_network = NeuralNetwork(hidden_layer_sizes = [100,100])
simple_network.forward_pass(train_images[0])
for x in simple_network.a:
    print(x.shape)

print(simple_network.a[-1])
print(simple_network.z[-1])

print(sum(softmax(simple_network.z[-1])))


(100,)
(100,)
(10,)
[0.0961335  0.10115931 0.10241098 0.10389784 0.10185598 0.0999216
 0.10112003 0.10048259 0.09940385 0.09361433]
[-5.06249054e-02  3.33811916e-04  1.26311869e-02  2.70453893e-02
  7.19708589e-03 -1.19769172e-02 -5.45757242e-05 -6.37831746e-03
 -1.71718769e-02 -7.71793341e-02]
1.0
